In [1]:
import numpy as np

In [ ]:
with open("../data/day18.txt") as f:
    data = f.readlines()

coordinates = [tuple(map(int, row.strip().split(","))) for row in data]

size = 71
grid = np.full((size, size), ".", dtype=str)
for x, y in coordinates[:1024]:
    grid[y, x] = "#"
grid

In [3]:
most_efficient_states = {}


directions = [
    np.array((-1, 0)),
    np.array((0, -1)),
    np.array((1, 0)),
    np.array((0, 1)),
]


def is_outside_bounds(test_position, size):
    return (
        test_position[0] == -1
        or test_position[1] == -1
        or test_position[0] == size[0]
        or test_position[1] == size[1]
    )


def find_possible_directions(arr, current_position):
    possible_directions = []
    for possible_step in directions:
        possible_next_position = tuple(current_position + possible_step)
        if (
            not is_outside_bounds(possible_next_position, arr.shape)
            and arr[possible_next_position] != "#"
        ):
            possible_directions.append(possible_step)
    return possible_directions


temp_arr = grid.copy()
starting_position = (0, 0)

current_position = tuple(starting_position)
current_route = [current_position]
routes_in_consideration = [current_route]
all_routes = []
all_scores = []

while True:
    try:
        current_route = routes_in_consideration.pop(0)
    except IndexError:
        break
    current_position = current_route[-1]
    possible_directions = find_possible_directions(temp_arr, current_position)
    for step in possible_directions:
        new_position = tuple(current_position + step)
        if new_position in current_route:
            continue  # Do not walk in loops
        new_route = current_route + [new_position]
        new_score = len(new_route)

        if new_score >= most_efficient_states.get(new_position, np.inf):
            continue
        if new_position == (size - 1, size - 1):
            all_routes.append(new_route)
            all_scores.append(len(new_route))
        else:
            most_efficient_states[new_position] = new_score
            routes_in_consideration.append(new_route)

In [ ]:
min(all_scores) - 1  # count steps

# Part 2: blocking byte

In [23]:
def grid_after_n_seconds(n, size, coordinates):
    grid = np.full((size, size), ".", dtype=str)
    for x, y in coordinates[:n]:
        grid[y, x] = "#"
    return grid


def can_solve_maze(maze):
    most_efficient_states = {}
    starting_position = (0, 0)

    current_position = tuple(starting_position)
    current_route = [current_position]
    routes_in_consideration = [current_route]
    all_routes = []
    all_scores = []

    while not all_routes:
        try:
            current_route = routes_in_consideration.pop(0)
        except IndexError:
            break
        current_position = current_route[-1]
        possible_directions = find_possible_directions(maze, current_position)
        for step in possible_directions:
            new_position = tuple(current_position + step)
            if new_position in current_route:
                continue  # Do not walk in loops
            new_route = current_route + [new_position]
            new_score = len(new_route)

            if new_score >= most_efficient_states.get(new_position, np.inf):
                continue
            if new_position == (size - 1, size - 1):

                all_routes.append(new_route)
                all_scores.append(len(new_route))
            else:
                most_efficient_states[new_position] = new_score
                routes_in_consideration.append(new_route)
    return bool(all_routes)


def find_first_true(func, options, low=None, high=None):
    """Finds the first option where func evaluates to True, performing binary search

    Parameters
    ----------
    func : callable
        Will be called with the option selected from options. func(options[0]) must be True,
        func(options[-1]) must be False
    options : Iterable
        A subscriptable iterable of options for which the lowest must be found where func evaluates to True
        It must be sorted in a way that before a certain index, func always evaluates to False, and afterwards always to True
    low : the highest known index for which func(option[index]) evaluates to True
    high : the lowest known index for which func(option[index]) evaluates to False

    Returns
    -------
    The first index for which func(options[index]) returns True

    """
    if low is None:
        if func(options[0]):
            return 0
        return find_first_true(func, options, 0, high)
    if high is None:
        high = len(options)
        if func(options[-1]):
            return find_first_true(func, options, low, high)
        raise RuntimeError("No True option found")
    mid = (low + high) // 2

    if low == mid:
        return low
    if func(options[mid]):
        return find_first_true(func, options, low, mid)
    return find_first_true(func, options, mid, high)

In [ ]:
def maze_unsolvable_after_n_seconds(n):
    temp_arr = grid_after_n_seconds(n, 71, coordinates)
    return not can_solve_maze(temp_arr)


n_seconds = find_first_true(
    maze_unsolvable_after_n_seconds, range(len(coordinates)), low=1024
)
coordinates[n_seconds]

In [26]:
# My actual submission took ~5 minutes brute forcing before I wrote the binary search
#
# for n in tqdm.tqdm(range(1024, len(coordinates))):

#     temp_arr = grid_after_n_seconds(n, 71, coordinates)


#     most_efficient_states = {}

#     starting_position = (0, 0)


#     current_position = tuple(starting_position)

#     current_route = [current_position]

#     routes_in_consideration = [current_route]


#     all_routes = []

#     all_scores = []


#     while not all_routes:

#         try:

#             current_route = routes_in_consideration.pop(0)

#         except IndexError:

#             break


#         current_position = current_route[-1]

#         possible_directions = find_possible_directions(temp_arr, current_position)

#         for step in possible_directions:

#             new_position = tuple(current_position + step)

#             if new_position in current_route:

#                 continue  # Do not walk in loops


#             new_route = current_route + [new_position]
#             new_score = len(new_route)

#             if new_score >= most_efficient_states.get(new_position, np.inf):
#                 continue

#             if new_position == (size - 1, size - 1):

#                 all_routes.append(new_route)

#                 all_scores.append(len(new_route))

#             else:

#                 most_efficient_states[new_position] = new_score

#                 routes_in_consideration.append(new_route)

#     if not all_routes:
#         print(n)

#         break